In [86]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [89]:
batch_size = 64
block_size = 64
max_iters = 200
eval_interval = 50
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 100
n_embd = 18
n_head = 3
n_layer = 4
dropout = 0.2

In [90]:
torch.manual_seed(1337)

In [91]:
with open('text.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [92]:
# chars = sorted(list(set(text.split())))
# vocab_size = len(chars)
# # vocab_size
# # create a mapping from characters to integers
# stoi = { ch:i+1 for i,ch in enumerate(chars) }
# itos = { i+1:ch for i,ch in enumerate(chars) }
# stoi[' '] = 0
# itos[0] = ' '
# encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
# decode = lambda l: ''.join([itos[i] for i in l])

# create a mapping from words to integersyou know Caius
# stoi = { ch:i for i,ch in enumerate(chars) }
# itos = { i:ch for i,ch in enumerate(chars) }
# encode = lambda s: [stoi[c] for c in s.split()] # encoder: take a string, output a list of integers
# decode = lambda l: ''.join([itos[i] for i in l])

# CREATE TABLE users (
#     user_id INT PRIMARY KEY,
#     username VARCHAR(50) NOT NULL,
#     email VARCHAR(100) UNIQUE,
#     password VARCHAR(100) NOT NULL,
#     first_name VARCHAR(50),
#     last_name VARCHAR(50),
#     age INT
#    

# );

words = ["user_id", "username", "email", "password", "first_name", "last_name", "age"]
vocab_size = len(words)
stoi = { ch:i for i,ch in enumerate(words) }
itos = { i:ch for i,ch in enumerate(words) }
stoi['-'] = vocab_size
itos[vocab_size] = '-'
stoi[' '] = vocab_size+1
itos[vocab_size+1] = ' '
vocab_size += 2
print(stoi)
print(itos)
def encode(s):
    res = []
    for i in s.split():
        for j in i.split('-'):
            res.append(stoi[j])
            res.append(stoi['-'])
        res.append(stoi[' '])
    return res
def decode(l):
    res = []
    for i in l:
        res.append(itos[i])
    return ''.join(res)

{'user_id': 0, 'username': 1, 'email': 2, 'password': 3, 'first_name': 4, 'last_name': 5, 'age': 6, '-': 7, ' ': 8}
{0: 'user_id', 1: 'username', 2: 'email', 3: 'password', 4: 'first_name', 5: 'last_name', 6: 'age', 7: '-', 8: ' '}


In [93]:
vocab_size

9

In [95]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [96]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [97]:
@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [98]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x) 
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1) 
        wei = self.dropout(wei)
        v = self.value(x) 
        out = wei @ v 
        return out

In [99]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [100]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [101]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [102]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx) 
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) 
        x = tok_emb + pos_emb
        x = self.blocks(x) 
        x = self.ln_f(x) 
        logits = self.lm_head(x) 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1) 
            idx_next = torch.multinomial(probs, num_samples=1) 
            idx = torch.cat((idx, idx_next), dim=1) 
        return idx

In [103]:
model = GPTLanguageModel()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

0.017793 M parameters


In [105]:
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(model)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 2.2131, val loss 2.2133
step 50: train loss 1.8810, val loss 1.8810
step 100: train loss 1.6386, val loss 1.6384
step 150: train loss 1.4585, val loss 1.4589
step 199: train loss 1.3403, val loss 1.3406


In [106]:
torch.save(model.state_dict(), 'query_forecast_state.pth')
torch.save(model, 'query_forecast.pth')

In [109]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
open('output.txt', 'w').write(decode(m.generate(context, max_new_tokens=1000)[0].tolist()))

4283

In [108]:
input_str = "c fcab beafc cfde fdae baedc c dbcaf "
context = torch.tensor(encode(input_str), dtype=torch.long, device=device).unsqueeze(0)
open('output2.txt', 'w').write(input_str + decode(m.generate(context, max_new_tokens=1000)[0].tolist()))

KeyError: 'c'